# CH8. 성능 최적화

- 실습해볼 task : Sentence Positive/Negative Classificationn using CNN(CNN 기반 문장의 긍부정 분류)
- 실습 순서
    - [참고자료](https://github.com/graykode/nlp-tutorial/blob/master/2-1.TextCNN/TextCNN.py)

## 0. 필요한 라이브러리 & 모듈

In [13]:
import pprint

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

from models import text_cnn

## 1. CH9장 내용 기반 데이터 전처리

- 데이터(일종의 corpus) load

In [2]:
corpus = [
            "i love you", 
            "he loves me", 
            "she likes baseball", 
            "i hate you", 
            "sorry for that", 
            "this is awful"
]
labels = [1, 1, 1, 0, 0, 0] 

- Tokenization 진행
    - 문장을 단어 단위로 분리

In [6]:
words = " ".join(corpus).split()
words = list(set(words))
words


['baseball',
 'hate',
 'you',
 'this',
 'sorry',
 'that',
 'loves',
 'i',
 'he',
 'for',
 'is',
 'she',
 'awful',
 'love',
 'likes',
 'me']

- 토큰에 정수 매핑하는 딕셔너리 만들기
    - 단어 데이터를 모델이 이해할 수 있는 정수로 이루어진 벡터로 바꾸기 위해

In [5]:
word_dict = {w: i for i, w in enumerate(words)}
word_dict

{'baseball': 0,
 'hate': 1,
 'you': 2,
 'this': 3,
 'sorry': 4,
 'that': 5,
 'loves': 6,
 'i': 7,
 'he': 8,
 'for': 9,
 'is': 10,
 'she': 11,
 'awful': 12,
 'love': 13,
 'likes': 14,
 'me': 15}

- input 데이터 텐서화
    - 문장 -> 벡터

In [9]:
inputs = torch.LongTensor([np.asarray([word_dict[n] for n in sentence.split()]) for sentence in corpus])
targets = torch.LongTensor([out for out in labels]) 
print(f"input 데이터 shape: {inputs.size()}")
print(f"target 데이터 shape: {targets.size()}")

input 데이터 shape: torch.Size([6, 3])
target 데이터 shape: torch.Size([6])


## 2. 성능 최적화 이용 학습
- 하이퍼파라미터 성능 최적화
    - 배치 정규화
    - 드롭아웃
    - 조기 종료
    - 모델 파리미터 튜닝
- 성능 최적화
    - 앙상블 사용

- check vanilla model

In [19]:
num_filters = 3 
filter_sizes = [2, 2, 2] 
vocab_size = len(word_dict)
embedding_size = 2 
sequence_length = 3 
num_classes = 2 
model = text_cnn.TextCNN(
    num_filters, filter_sizes, vocab_size,
    embedding_size, sequence_length, num_classes
)

In [20]:
from train import train_model

In [21]:
train_model(model,inputs,targets,100)
test_text = 'he hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = torch.LongTensor(tests)

# Predict
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

Epoch: 0001 cost = 0.689684
Epoch: 0002 cost = 0.686636
Epoch: 0003 cost = 0.683598
Epoch: 0004 cost = 0.680571
Epoch: 0005 cost = 0.677553
Epoch: 0006 cost = 0.674546
Epoch: 0007 cost = 0.671548
Epoch: 0008 cost = 0.668557
Epoch: 0009 cost = 0.665572
Epoch: 0010 cost = 0.662590
Epoch: 0011 cost = 0.659609
Epoch: 0012 cost = 0.656628
Epoch: 0013 cost = 0.653645
Epoch: 0014 cost = 0.650659
Epoch: 0015 cost = 0.647669
Epoch: 0016 cost = 0.644673
Epoch: 0017 cost = 0.641670
Epoch: 0018 cost = 0.638661
Epoch: 0019 cost = 0.635644
Epoch: 0020 cost = 0.632619
Epoch: 0021 cost = 0.629586
Epoch: 0022 cost = 0.626543
Epoch: 0023 cost = 0.623492
Epoch: 0024 cost = 0.620431
Epoch: 0025 cost = 0.617360
Epoch: 0026 cost = 0.614280
Epoch: 0027 cost = 0.611218
Epoch: 0028 cost = 0.608128
Epoch: 0029 cost = 0.605034
Epoch: 0030 cost = 0.601938
Epoch: 0031 cost = 0.598830
Epoch: 0032 cost = 0.595710
Epoch: 0033 cost = 0.592578
Epoch: 0034 cost = 0.589436
Epoch: 0035 cost = 0.586282
Epoch: 0036 cost = 0